In [81]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import numpy as np
import pandas as pd
from torch.utils.data import Dataset

class Model(nn.Module):

    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(132, 200)
        self.linear2 = nn.Linear(200, 200)
        self.linear3 = nn.Linear(200, 500)
        self.linear4 = nn.Linear(500, 200)
        self.output = nn.Linear(200, 2)
    def forward(self, x):
        
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = F.relu(self.linear2(x))
        return F.softmax(self.output(x), dim=1)
    

class SticksDataset(Dataset):
    def __init__(self, sticks_path, labels_path): 
        self.sticks = torch.tensor(np.load(sticks_path)).float()

        
        self.labels = torch.tensor(np.load(labels_path)).float()

    def __len__(self):
        return self.labels.shape[0]

    def __getitem__(self, idx):
        return self.sticks[idx], self.labels[idx]

In [82]:
from torch.utils.data import DataLoader
data = SticksDataset("sticks.npy", "labels.npy")
train_dataloader = DataLoader(data, batch_size=64, shuffle=True)

In [89]:
import torch.optim as optim
model = Model()

criter = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5)
nttlstps = len(data)
model.train()
for epoch in range(200):
    running_loss = 0.0
    for x, (train_features, train_labels) in enumerate(train_dataloader):
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = model(train_features)
        loss = criter(outputs, train_labels)
        loss.backward()
        optimizer.step()

    # print statistics
        running_loss += loss.item()
    if epoch % 10 == 0:
        print(f'[epoch: {epoch}] loss: {loss}')
    
    
    running_loss = 0.0

[epoch: 0] loss: 0.22794899344444275
[epoch: 10] loss: 0.024764686822891235
[epoch: 20] loss: 0.004781671799719334
[epoch: 30] loss: 0.0007192490738816559
[epoch: 40] loss: 0.01733780838549137
[epoch: 50] loss: 0.01762513816356659
[epoch: 60] loss: 0.0004258677363395691
[epoch: 70] loss: 0.015977567061781883
[epoch: 80] loss: 0.016008982434868813
[epoch: 90] loss: 0.015954524278640747
[epoch: 100] loss: 9.663841046858579e-05
[epoch: 110] loss: 8.667635847814381e-05
[epoch: 120] loss: 0.015879176557064056
[epoch: 130] loss: 0.031709618866443634
[epoch: 140] loss: 3.6911053030053154e-05
[epoch: 150] loss: 0.029532723128795624
[epoch: 160] loss: 0.00011608311615418643
[epoch: 170] loss: 0.00018017698312178254
[epoch: 180] loss: 0.00014380984066519886
[epoch: 190] loss: 0.0006116138538345695


In [92]:
import torch
test_data = SticksDataset("test_datas.npy", "test_labels.npy")
test_dataloader = DataLoader(test_data, batch_size=1, shuffle=True)
total = 0
correct = 0
model.eval()
with torch.no_grad():

    for x, (test_features, test_labels) in enumerate(test_dataloader):
            # forward + backward + optimize
            
        outputs = model(test_features)
        if torch.argmax(test_labels) == torch.argmax(outputs):
            correct += 1
            
        total +=1
        
print(correct/total)

0.744421906693712


In [91]:
model_scripted = torch.jit.script(model) # Export to TorchScript
model_scripted.save('model_scripted_1.pt') # Save